In [ ]:
%pip install semantic-router==0.0.37
%pip install sentence-transformers
%pip install chromadb
%pip install pypdf
%pip install langchain-community

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

encoder.encode("Hello, World!")

In [2]:
import chromadb

client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb",
)

collection = client.create_collection("skin-cancer")

In [13]:
import chromadb.utils.embedding_functions as embedding_functions
chroma_encode = embedding_functions.CohereEmbeddingFunction(
    api_key=os.environ["COHERE_API_KEY"], model_name="embed-english-v3.0"
)

Document(metadata={'source': '/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/documents/Actinic keratosis - Diagnosis & treatment - Mayo Clinic.pdf', 'page': 0}, page_content="Actinic keratosis\nDiagnosis\nYour health care provider will likely be able to determine whether you have an actinic\nkeratosis simply by looking at your skin. If there's any doubt, your health care\nprovider may do other tests, such as a skin biopsy. During a skin biopsy, a small\nsample of skin is taken for analysis in a lab. A biopsy can usually be done in a clinic\nafter a numbing injection.\nEven after treatment for actinic keratosis, your health care provider might suggest\nthat you have your skin checked at least once a year for signs of skin cancer.\nTreatment\nAn actinic keratosis sometimes disappears on its own but might return after more\nsun exposure. It's hard to tell which actinic keratoses will develop into skin cancer, so\nthey're usually removed as a precaution.\nMedicines\nIf you ha

In [26]:
document = ""
for doc in documents:
    document += doc.page_content + "\n"

In [34]:
# semantic chunking
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
import os
from semantic_router.encoders import CohereEncoder

os.environ["COHERE_API_KEY"] = "kUYLefL4SNo36iNh5tF2H1gGi5XfZyTbOBJNknt1"
logger.setLevel("WARNING")
encoder = CohereEncoder()
splitter = RollingWindowSplitter(
    encoder=encoder,
    min_split_tokens=50,
    max_split_tokens=400,
    window_size=2,
    plot_splits=True,
    enable_statistics=True,
)

In [52]:
from pypdf import PdfReader
import os

pdf_paths = os.listdir(
    "/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/documents"
)
documents = PdfReader(f"{path}/{pdf_paths[0]}")
document = ""
for doc in documents.pages:
    document += doc.extract_text() + "\n"

splits = splitter([document])
" ".join(splits[0].docs)

2024-08-03 08:53:16 WARNING semantic_router.utils.logger Plotting is disabled. Please `pip install semantic-router[processing]`.


Splitting Statistics:
  - Total Documents: 120
  - Total Splits: 9
  - Splits by Threshold: 8
  - Splits by Max Chunk Size: 0
  - Last Split: 1
  - Minimum Token Size of Split: 53
  - Maximum Token Size of Split: 252
  - Similarity Split Ratio: 0.89


"Actinic keratosis Diagnosis Your health care provider will likely be able to determine whether you have an actinic keratosis simply by looking at your skin. If there's any doubt, your health care provider may do other tests, such as a skin biopsy. During a skin biopsy, a small sample of skin is taken for analysis in a lab. A biopsy can usually be done in a clinic after a numbing injection. Even after treatment for actinic keratosis, your health care provider might suggest that you have your skin checked at least once a year for signs of skin cancer. Treatment An actinic keratosis sometimes disappears on its own but might return after more sun exposure. It's hard to tell which actinic keratoses will develop into skin cancer, so they're usually removed as a precaution."

In [42]:
print(" ".join(splits[0].docs))

Actinic keratosis Diagnosis Your health care provider will likely be able to determine whether you have an actinic keratosis simply by looking at your skin. If there's any doubt, your health care provider may do other tests, such as a skin biopsy. During a skin biopsy, a small sample of skin is taken for analysis in a lab. A biopsy can usually be done in a clinic after a numbing injection. Even after treatment for actinic keratosis, your health care provider might suggest that you have your skin checked at least once a year for signs of skin cancer. Treatment An actinic keratosis sometimes disappears on its own but might return after more sun exposure. It's hard to tell which actinic keratoses will develop into skin cancer, so they're usually removed as a precaution.
